# Learning to Feel; Training the Model
*Curtis Miller*

Now that we have data, let's attempt to train a model for predicting emotions from images. Let's first load in the data.

In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib
from sklearn.metrics import classification_report, accuracy_score
%matplotlib inline

In [ ]:
faces = pd.read_csv("fer2013.csv")

Now let's split into training and testing data. We will use the suggested categories provided with the dataset, and use the private test set rather than the public test set.

In [ ]:
facegroups = faces.groupby("Usage")
face_train = facegroups.get_group("Training")
face_test = facegroups.get_group("PrivateTest")
face_train

Below is code from the previous video.

In [ ]:
emotion_code = {0: "angry",
                1: "disgust",
                2: "fear",
                3: "happy",
                4: "sad",
                5: "surprise",
                6: "neutral"}

def string_to_image(pixelstring):
    return np.array(pixelstring.split(' '), dtype=np.int16).reshape(48, 48)

For emotion detection we will use the LBPH recognizer that is supplied with OpenCV. Our classes are different emotions, and the classifier will learn to group and separate faces based on these classes.

In [ ]:
emotion_detector = cv2.face.LBPHFaceRecognizer_create()
emotion_detector.train(face_train.pixels.map(string_to_image), face_train.emotion.as_matrix())

How does this classifier do on the training set?

In [ ]:
plt.imshow(string_to_image(face_train.pixels[8]))

In [ ]:
emotion_detector.predict_label(string_to_image(face_train.pixels[8]))

It handles one face well. Let's see how it does on training data. We won't use the entire dataset simply because it would take too long on my laptop.

In [ ]:
# Estimated to take 10 minutes on my laptop
train_emotion_predicted = face_train.pixels[:100].map(lambda x: emotion_detector.predict_label(string_to_image(x)))
train_emotion_predicted

In [ ]:
print(classification_report(face_train.emotion[:100], train_emotion_predicted))

This report is too good to be true. It's not surprising considering that classification using LBPH shares a lot in common with nearest neighbor approaches; it will do perfectly on the training set as a result.

If we wanted a better sense of how well the algorithm would do, we should employ cross-validation. However, that would be too time consuming for this video.

So how does the algorithm do on test data?

In [ ]:
test_emotion_predicted = face_test.pixels[:100].map(lambda x: emotion_detector.predict_label(string_to_image(x)))
print(classification_report(face_test.emotion[:100], test_emotion_predicted))

In [ ]:
print(accuracy_score(face_test.emotion[:100], test_emotion_predicted))

These results don't appear particularly encouraging. However, emotion recognition is a hard problem (and perhaps defined inappropriately). Our algorithm does do better than random guessing or simply guessing the most common label (the latter case would yield an accuracy of 22%). Since there's "overlap" between emotions, it's also possible that the algorithm guesses similar (but "incorrect") emotions.

So we may not have done too poorly.

(In the [Kaggle competition](https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/leaderboard) the best accuracy was 71%; our algorithm, if it maintained this accuracy, would place 31st out of 56, and we haven't done that much.)